In [ ]:
!bzip2 -d svt-2023.xml.bz2


In [ ]:
!pip3.10 uninstall spacy
# grep -o "<sentence" svt-2023.xml | wc -l


Found existing installation: spacy 3.8.4
Uninstalling spacy-3.8.4:
  Would remove:
    /Library/Frameworks/Python.framework/Versions/3.10/bin/spacy
    /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy-3.8.4.dist-info/*
    /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/*
Proceed (Y/n)? ^C


In [3]:
with open("svt-2023.xml", "r", encoding="utf-8") as f:
    for _ in range(5000):  # Read the first 10 lines
        print(f.readline().strip())


<?xml version='1.0' encoding='UTF-8'?>
<!--
Sparv version: 5.1.1dev0
Annotation date: '2023-08-28'
-->
<corpus id="svt-2023">
<articles>
<text authors="|Ellinor Rostedt|" data_collected="2023-02-08" date="2023-01-12T13:54:01+01:00" datefrom="20230112" dateto="20230112" timefrom="135401" timeto="135401" id="38147263" blingbring="|språkljud:144.997|olja:84.229|fetthalt:66.616|" swefn="|Proliferating_in_number:369.654|Measure_mass:217.988|Duplication:171.754|" _id="0b5834" lix="40.98" nk="2.28" ovix="58.26" section="Blekinge" title="Fördubblad elproduktion på Karlshamnsverket – på ett år" url="https://www.svt.se/nyheter/lokalt/blekinge/fordubblad-elproduktion-pa-karlshamnsverket-pa-ett-ar">
<paragraph _geocontext="|" type="lead">
<sentence _geocontext="|" id="0b58422">
<ne ex="ENAMEX" name="Karlshamnsverket" subtype="CRP" type="ORG">
<token blingbring="|" swefn="|" _tail="\s" lemma="|" complemgram="|" compwf="|Karlshamns+verket|" lex="|" dephead="2" deprel="SS" msd="NN.NEU.SIN.DEF.NOM" ms

In [18]:
import xml.etree.ElementTree as ET

xml_file = "svt-2023.xml"

count = 0
for event, elem in ET.iterparse(xml_file, events=("end",)):
    if elem.tag == "sentence":
        count += 1
        elem.clear()  # Free memory

print(f"Total sentences in the file: {count}")


Total sentences in the file: 495757


In [ ]:

import xml.etree.ElementTree as ET
import csv
from transformers import pipeline
import re
import json

# Load Swedish BERT NER model
nlp = pipeline("ner", model="KBLab/bert-base-swedish-cased-ner", tokenizer="KBLab/bert-base-swedish-cased-ner")

# Define input/output files
xml_file = "svt-2023.xml"
csv_file = "sentences_entities_bert.csv"

# Helper function to extract capitalized words as potential names
def extract_capitalized_words(sentence, found_entities):
    words = sentence.split()
    possible_names = []
    for i, word in enumerate(words):
        # If the word starts with an uppercase letter and is not the first word (to avoid sentence-start capitalization)
        if re.match(r"^[A-ZÅÄÖ][a-zåäö]+$", word) and i > 0:
            if word not in found_entities and len(word) > 3:
                print(f"Found possible name: {word}")
                possible_names.append(word)
    return possible_names

def clean_sentence(sentence: str) -> str:
    # Remove leading '-'
    sentence = re.sub(r'^-\s*', '', sentence)
    
    # Replace '-' with ','
    sentence = sentence.replace('–', ',')
    
    sentence = sentence.replace('”', '')
    
    # Remove all commas
    sentence = sentence.replace(',', '')
    
    # Replace ' : ' with ' ', but not when words are adjacent (e.g., SAKO:s remains unchanged)
    sentence = re.sub(r'(?<=\s):\s', ' ', sentence)
    
    # Remove occurrences of [ ... ]
    sentence = re.sub(r'\[.*?\]', '', sentence)
    
    # Replace middle quotation marks with a comma before the quoted word
    sentence = re.sub(r'(?<=\S)\s*["””]\s*(?=\S)', ',', sentence)
    
    return sentence

# Open CSV file for writing
with open(csv_file, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence", "entities", "metadata"])  # CSV headers

    # Parse XML
    context = ET.iterparse(xml_file, events=("start", "end"))
    count = 0

    for event, elem in context:
        if event == "end" and elem.tag == "sentence":
            # Extract sentence text
            words = [token.text for token in elem.findall(".//token") if token.text]
            sentence = " ".join(words)
            sentence = clean_sentence(sentence)
        
        
            # Run BERT NER model
            bert_entities = []
            current_entity = ""
            current_label = None
            previous_end = -1  # Tracks end position of the last processed token

            for token in nlp(sentence):
                word = token["word"]
                label = token["entity"]
                score = token["score"]
                start = token["start"]
                end = token["end"]

                if word.startswith("##"):
                    # Merge with previous token without space
                    current_entity += word[2:]
                else:
                    # If this token continues an entity (same label, consecutive in text), merge it
                    if current_entity and label == current_label and start == previous_end:
                        current_entity += word  # No space if directly consecutive
                    elif current_entity and label == current_label:
                        current_entity += " " + word  # Add space if it's part of the same entity
                    else:
                        # Store completed entity before starting a new one
                        if current_entity and current_label in ["PER", "ORG", "LOC", "EVN"]:
                            bert_entities.append(current_entity)

                        # Start a new entity if confidence is high
                        if score > 0.90 and label in ["PER", "ORG", "LOC", "EVN"]:
                            current_entity = word
                            current_label = label
                        else:
                            current_entity = ""
                            current_label = None

                previous_end = end  # Update last processed token end position

            # Store last entity
            if current_entity and current_label in ["PER", "ORG", "LOC", "EVN"]:
                bert_entities.append(current_entity)



            # Try to capture missing names using capitalized words
        
            all_entities_str = json.dumps(bert_entities, ensure_ascii=False) if bert_entities else ""

            
            # Write to CSV
            sentence = clean_sentence(sentence)
            metadata = [{"source": "SVT"}]
            row = [sentence, all_entities_str, metadata]
            writer.writerow(row)

            # Free memory
            elem.clear()

            count += 1
            if count >= 50:  # Limit to 50 sentences for testing
                break

print(f"✅ Processed first 500 sentences with BERT! Saved as {csv_file}.")


Some weights of the model checkpoint at KBLab/bert-base-swedish-cased-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Processed first 500 sentences with BERT! Saved as sentences_entities_bert.csv.


In [ ]:
from transformers import pipeline

nlp = pipeline("ner", model="KBLab/bert-base-swedish-cased-ner", tokenizer="KBLab/bert-base-swedish-cased-ner")

test_sentence = "Det är det andra tunga beskedet för Mörrum Hockey på kort tid, Hockey"

# Run NER
tokens = nlp(test_sentence)

# Print results
for token in tokens:
    print(token)


Some weights of the model checkpoint at KBLab/bert-base-swedish-cased-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entity': 'ORG', 'score': 0.92606175, 'index': 8, 'word': 'Mör', 'start': 36, 'end': 39}
{'entity': 'ORG', 'score': 0.90471256, 'index': 9, 'word': '##rum', 'start': 39, 'end': 42}
{'entity': 'ORG', 'score': 0.9758506, 'index': 10, 'word': 'Hockey', 'start': 43, 'end': 49}
{'entity': 'TME', 'score': 0.9996295, 'index': 11, 'word': 'på', 'start': 50, 'end': 52}
{'entity': 'TME', 'score': 0.9996668, 'index': 12, 'word': 'kort', 'start': 53, 'end': 57}
{'entity': 'TME', 'score': 0.9996687, 'index': 13, 'word': 'tid', 'start': 58, 'end': 61}
{'entity': 'PER', 'score': 0.95311457, 'index': 16, 'word': 'Hockey', 'start': 64, 'end': 70}
